<a href="https://colab.research.google.com/github/dnhshl/cc-ai/blob/main/Kryptographie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kryptographie

Beispiele u.a. in Anlehnung an [Practical Cryptography for Developers](https://cryptobook.nakov.com/)

## AES Encrypt / Decrypt

In [2]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00


In [3]:
from Crypto.Cipher import AES
import binascii, os

In [4]:
def encrypt_AES_GCM(msg, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (ciphertext, aesCipher.nonce, authTag)

def decrypt_AES_GCM(encryptedMsg, secretKey):
    (ciphertext, nonce, authTag) = encryptedMsg
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

In [5]:
secretKey = os.urandom(32)  # 256-bit random encryption key
print("Encryption key:", binascii.hexlify(secretKey))

Encryption key: b'73202abc47eaa94bdbee1917863ce98e69c51c1d635ca6bf0d6cbe111c79ce6d'


In [6]:
msg = b'Hello World, how are you today?'

In [7]:
encryptedMsg = encrypt_AES_GCM(msg, secretKey)
print("encryptedMsg", {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'aesIV': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2])
})

encryptedMsg {'ciphertext': b'21ef765318813f50524295e1eb6224fce2cb80f35bc43589b6e6cb33d3b81c', 'aesIV': b'ef705540cd9b85588fe5609c765e9552', 'authTag': b'38613b15b43c3467022c6c2c53b2cc73'}


In [8]:
decryptedMsg = decrypt_AES_GCM(encryptedMsg, secretKey)
print("decryptedMsg", decryptedMsg)

decryptedMsg b'Hello World, how are you today?'


## RSA

In [9]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii

### RSA Keys

In [26]:
keyPair = RSA.generate(3072)

pubKey = keyPair.publickey()
print(f"Public key:  (n={hex(pubKey.n)}, e={hex(pubKey.e)})")
pubKeyPEM = pubKey.exportKey()
print(pubKeyPEM.decode('ascii'))


privateKey = keyPair
print(f"Private key: (n={hex(pubKey.n)}, d={hex(keyPair.d)})")
privKeyPEM = keyPair.exportKey()
print(privKeyPEM.decode('ascii'))

Public key:  (n=0xb9248483cbb91838c144d547c120eb963ce5d422a8810b7621b56263049e7a4a6540b46390d08a723ace13415eb7e3d70563e4699f1cfdc65c71bc7da68aeadd063ed552af255551393304023596a4ebfa1835237d4290c068576b0fa3e244691987a549ffc1a5aea99fba9a87a88e45988290671e8660c2ec60bba560fb572c2c8f79661d14c875d57b22838dc5044135da3f77264f2808f760697b7fdb3d5e4edc62e6334f51229d15045d9967a28eaf2df2c8ce7818cb28d4cbed50663c8a828219dba8d398435244dba8f1b77274c4103840fd46d01c708cc9b17ef87f0db8f35d8729c73852c1ea05bb93251a0e913626163a57584550156a116af8536647ecb69cd3b7eb6d23e983bd933e285c16268b4010da2a573557aede854808d5b33aa456458228a89ef1e45b3a7605d0f07efa585fcd8a5401a6d7843ad54e6e89dfd229394198736aed90ce1c0a8627a620d5fee9c3a99555e0ac63d8c97a22ca26d614bdc38e2e33fb3590bbb8dd74bcb3a5ac381ed5e41bea5dc2e57d712d, e=0x10001)
-----BEGIN PUBLIC KEY-----
MIIBojANBgkqhkiG9w0BAQEFAAOCAY8AMIIBigKCAYEAuSSEg8u5GDjBRNVHwSDr
ljzl1CKogQt2IbViYwSeekplQLRjkNCKcjrOE0Fet+PXBWPkaZ8c/cZccbx9porq
3QY+1VKvJVVROTMEAjWWpOv6GDUjfUKQwGhXaw+j4kRp

### RSA Encrypt / Decrypt

In [27]:
msg = b'A message for encryption'
encryptor = PKCS1_OAEP.new(pubKey)
encrypted = encryptor.encrypt(msg)
print("Encrypted:", binascii.hexlify(encrypted))

Encrypted: b'a48f89bde1038e9cc962b18646468ffffb078ab530abe94531e0d817d365f5c89cadadf0c28dc19c94a5d6656d74077eae03a7f6c493c794e421354c9841df6f6d7ba9eac586bf20549ba6da6987e26c8e47ccfc008523d999060b5cf3cb63694795fc403a6b269a5dc8fa035a5e6529f4ed9641a48f4eb124bdb5e250804345d5e8197b548b1f7506e74ac0185e45b3c3302b8e29b93a133183dbbd25871f956f6016c8581617e124b86114ba05ce1c1eb4228398905df10b89d6ba9f429d8b9d849823b371272ba777856f1d70ecd2e0befd1cc0c01d81aa7ed78be6ddce6ba01aa620c530783b992fe82b5867fdbae059499842f4752c6455be58ace9558b846bded4461c6c7abed070e136da7b54234c52034709609e5417601e99c46f1fc2757811087487d113cfd4d7e2760caabf8f84c40150b2dc827a2400f8d1863dc92dc46fb92c161bcb85b67eec831e377f16f3a1f0f588836ad0d873e2cdd1810025e62a0b38b528396427bf45c92e26a864b6a671539e2555d218c53e1151ac'


In [29]:
decryptor = PKCS1_OAEP.new(privateKey)
decrypted = decryptor.decrypt(encrypted)
print('Decrypted:', decrypted)

Decrypted: b'A message for encryption'


## Hashing

In [38]:
import hashlib, binascii
from Crypto.Hash import SHA256
from Crypto.Hash import SHA3_256

In [40]:
text = 'Überweise 100 Euro'
msg = text.encode("utf8")

md5hash = hashlib.md5(msg).digest()
print("MD5:   ", binascii.hexlify(md5hash))

sha256hash = SHA256.new(msg).digest()
print("SHA-256:   ", binascii.hexlify(sha256hash))

sha3_256 = SHA3_256.new(msg).digest()
print("SHA3-256:  ", binascii.hexlify(sha3_256))



MD5:    b'8e222fd7af1c844f25ba54f813eb3240'
SHA-256:    b'b3f67f3b2929597cbe9ddaa6e88ffe8bfd2e898dfd00bda71e618a02ed6b39af'
SHA3-256:   b'f980913d0c933e061b87f5dd62397d01c9f0ac9653c66aee9b6aca3805ea3903'


In [41]:
text = 'Überweise 1000 Euro'
msg = text.encode("utf8")

md5hash = hashlib.md5(msg).digest()
print("MD5:   ", binascii.hexlify(md5hash))

sha256hash = SHA256.new(msg).digest()
print("SHA-256:   ", binascii.hexlify(sha256hash))

sha3_256 = SHA3_256.new(msg).digest()
print("SHA3-256:  ", binascii.hexlify(sha3_256))


MD5:    b'4d5e112077c5c036328ffe9262ad8b24'
SHA-256:    b'9bb4fc8e9b6475c9a8850a07c67c509e69cad63c7e6215ff783d62d6d21b2535'
SHA3-256:   b'06702a278e4b9a0a638df0e090055352faee4217a6016b33dac250a13cbf40db'


## MAC

In [20]:
import hashlib, hmac, binascii

In [19]:
def hmac_sha256(key, msg):
  return hmac.new(key, msg, hashlib.sha256).digest()

key = b"12345"


In [25]:
text = 'Überweise 1000 Euro'
msg = text.encode("utf8")

print(binascii.hexlify(hmac_sha256(key, msg)))

b'9db590d0d75dcb2a9395283cb78ddb3786869e2231dc818b59c80cf75f837a49'


## Signatur


In [35]:
from Crypto.Signature import pkcs1_15


### Alice

In [42]:
text = 'Überweise 1000 Euro'
msg = text.encode("utf8")

# Hash
hash_obj = SHA3_256.new(msg)
print("SHA-256:   ", binascii.hexlify(hash_obj.digest()))

# Encrypt Hash with private key
signature = pkcs1_15.new(privateKey).sign(hash_obj)
print("Signature", binascii.hexlify(signature))


SHA-256:    b'06702a278e4b9a0a638df0e090055352faee4217a6016b33dac250a13cbf40db'
Signature b'477751860d24218c7c0eede1fba72b7977559f4778540ee7ab4eca87c08fb17f013c1ca7228e930a398ff21333b9b0e2bbf3a281c0d3e4ae36d0f46ce0fa211af1cf0c3b597687e3c620c29b1d7d5e58899599b943b16b3facb0eb6339f6924a4e3dd9ca49b68219999b62ef0efc3008dbb3e42339c26d3bdaee7fc2b7a92f9e4c36506d3ff47eb57ac5f210760b254af9a4e0a02f2c20da524d7ed5513e4962a34673903eb2431c43859c0321840e6f2f8837428b805b08fb5bc745cd63bf3eb448f47027d977d6fdca42eb6129ff60f38e12192b1dd6e93a40847519e057b4108947ee9dcced9d664280d2d371b32761de603e96367cf89c266afa530d57ec077161e940924c75cff31fe8cd52680ec61627ec985d48f14e17951f0af1dd91ec1b2134e4191c9cd16bd7356475067d336af8a8632c30e493c79f1a1f52519f1b71b6cb9f4642b30ae9e195d120921d85e46a1a6a64e2a2bb89fb438f0931bdfa1a93ac538079617728411cd6f4b53dc9eee9c7efcb39a8187b19b242009439'


### Bob

In [43]:
msg = msg

# Hash
hash_obj = SHA3_256.new(msg)

# Verify the signature with the public key
try:
    pkcs1_15.new(pubKey).verify(hash_obj, signature)
    print("The signature is valid.")
except (ValueError, TypeError):
    print("The signature is not valid.")

The signature is valid.


## Verschlüsseln und Signieren

In [48]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
import binascii

In [49]:

# Alice generates her RSA key pair
alice_key = RSA.generate(2048)
alice_private_key = alice_key
alice_public_key = alice_key.publickey()

# Bob generates his RSA key pair
bob_key = RSA.generate(2048)
bob_private_key = bob_key
bob_public_key = bob_key.publickey()

### Alice

In [50]:

# Message to be signed and encrypted
message = b'This is a secret message'

# Alice signs the message with her private key
hash_obj = SHA256.new(message)
signature = pkcs1_15.new(alice_private_key).sign(hash_obj)

# Alice encrypts the message and the signature with Bob's public key
encryptor = PKCS1_OAEP.new(bob_public_key)
encrypted_message = encryptor.encrypt(message)
print("Encrypted Message:", binascii.hexlify(encrypted_message))


Encrypted: b'b9d89a3ddaf00eb7f168e44dea0e845c2dd4e39006fcc9442ae40c5a90909f9e523bca4c05077a494191da2c9409512b9e3126b1ac7d2a11976c095e3a4a47588150346e4a7f9f012d71aea17db62cd56d9342d806288155fdee1a8af6379db84c1c91875000f611faad093b9d3871f8005056ec41ec3ce1574109f39a9ac0e8e4d2ca6af1901494a7ec787651c8183e838ead8675a755ff38c4e3c9ad867edabe37e45a573d4d1f7a0882333645e249f22d3887e4663f6219f19438a02d9c4138083c7269667b43d9e588587f4d010a0c57276ba33a4a0bff1d51bbde4551917ec28e6f19927df9d8ae4ccf37a27e990326e577ac3d0d3a8743670916bc4558'


### Bob

In [51]:
# Bob decrypts the received message and signature with his private key
decryptor = PKCS1_OAEP.new(bob_private_key)
decrypted_message = decryptor.decrypt(encrypted_message)
print('Decrypted Message:', decrypted_message)

# Bob verifies the signature with Alice's public key
hash_obj = SHA256.new(decrypted_message)
try:
    pkcs1_15.new(alice_public_key).verify(hash_obj, signature)
    print("The signature is valid.")
except (ValueError, TypeError):
    print("The signature is not valid.")


Decrypted: b'This is a secret message'
The signature is valid.
